Test Connection

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
elastic_password = os.getenv("ELASTIC_PASSWORD")
from elasticsearch import Elasticsearch

es = Elasticsearch(
    ["http://127.0.0.1:9200"],
    http_auth=("elastic", elastic_password)
)

response = es.cluster.health()
print("Cluster Health:")
print(response)

Cluster Health:
{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 33, 'active_shards': 33, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'unassigned_primary_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


C:\Users\paulc\AppData\Local\Temp\ipykernel_39416\429568990.py:13: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Populate

In [1]:
import json
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

# Load environment variables
load_dotenv()
elastic_password = os.getenv("ELASTIC_PASSWORD")

# Connect to Elasticsearch
es = Elasticsearch(
    ["http://127.0.0.1:9200"],
    http_auth=("elastic", elastic_password)
)

# Check cluster health
response = es.cluster.health()
print("Cluster Health:", response)

# Define index name
index_name = "faqs"

# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "Question": {
                "type": "text",
                "analyzer": "standard"
            },
            "Answer": {
                "type": "text",
                "analyzer": "standard"
            }
        }
    }
}

# Check if index exists, create if not
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created with mapping.")
else:
    print(f"Index '{index_name}' already exists.")


# Load the JSON data
with open("general_faqs.json", "r", encoding="utf-8") as file:
    faqs = json.load(file)

# Ensure index exists (create if not)
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Insert documents into Elasticsearch
for i, faq in enumerate(faqs):
    es.index(index=index_name, id=i+1, body=faq)

print(f"Inserted {len(faqs)} FAQs into Elasticsearch.")

# Verify by retrieving one document
doc = es.get(index=index_name, id=1)
print("\nSample Document:")
print(doc["_source"])


C:\Users\paulc\AppData\Local\Temp\ipykernel_51328\4103639736.py:11: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 33, 'active_shards': 33, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'unassigned_primary_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}
Index 'faqs' created with mapping.
Inserted 249 FAQs into Elasticsearch.

Sample Document:
{'Question': 'What is financial difficulty?', 'Answer': 'Put simply, financial difficulty is when you can’t pay your bills. It can happen as a result of a job loss, illness, family break ups, natural disasters like droughts or floods and many other reasons.If you can’t make Suncorp Bank loan repayments or meet other financial commitments – or you think you’ll be experiencing difficulty soon – we’re here to help.Please contact us on1800